In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install dython
from dython.model_utils import metric_graph
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import KBinsDiscretizer
#from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
import shap




In [ ]:
#%%
df= pd.read_csv(r'../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head(2)

In [ ]:

from dython import nominal

import warnings
warnings.filterwarnings('ignore')

a= nominal.associations(df,figsize=(30,30),mark_columns=True);


In [ ]:
cols= df.select_dtypes('object').columns
to_eliminate= ['customerID','TotalCharges' , 'Churn']
new_cols = [col for col in cols if col not in to_eliminate]
new_cols

In [ ]:
df_new= df.copy()
df_new['Churn']= df_new['Churn'].replace({'Yes':1, 'No':0}).astype('float')
for col in new_cols:
    df_new.groupby(col)['Churn'].mean().plot(kind='bar')
    plt.show();
   

In [ ]:

#%%
class MyOneHotEncoder(TransformerMixin, BaseEstimator):
    
    def __init__(self, cat_cols=['tenure', 'MonthlyCharges', 'TotalCharges']):
        super().__init__()
        self.cat_cols= cat_cols  
        self.hotcoder= ce.OneHotEncoder (cols=self.cat_cols, use_cat_names=True)
        
    def fit(self, X_train, y=None):
        self.hotcoder.fit(X_train)

        return self
    
    def transform(self, X):
        X_new= self.hotcoder.transform(X)
        self.new_cols=  X_new.columns
        return X_new

#%%
class RandomForestFiller (TransformerMixin, BaseEstimator):

    def __init__ (self, max_depth= None):
        super().__init__()
        self.max_depth= max_depth
        self.rfc= RandomForestRegressor(max_depth=self.max_depth)
    
    def fit(self, X, y=None):
        X= X.replace(' ', np.nan).dropna(subset=['TotalCharges'])
        y= X['TotalCharges'] # totalcharges
        X= X.drop(['TotalCharges'], axis=1)
        self.rfc.fit(X, y)
        return self
    
    def transform (self,X):
        X_copy= X.copy().drop(['TotalCharges'], axis=1)
        filled_TotalCharges= self.rfc.predict(X_copy)
        X_copy['TotalCharges']= X['TotalCharges'].replace(' ', np.nan)
        X_copy['TotalCharges']= X_copy['TotalCharges'].combine_first(pd.Series(filled_TotalCharges, index= X_copy.index))
        return X_copy


#%%

class BinsEncoder(TransformerMixin, BaseEstimator):
    
    def __init__(self, k=5):
        super().__init__()
        self.k= k
        self.df_numeric_to_bins=KBinsDiscretizer(n_bins=self.k, encode='ordinal', strategy='quantile',)
        
        
    def fit(self, X, y=None):
        #cols = X.select_dtypes([np.number]).columns
        self.cols=['tenure', 'MonthlyCharges', 'TotalCharges']
        self.df_numeric_to_bins.fit(X[self.cols])
               
        return self
    
    def transform(self, X):
        new_binned_cols = pd.DataFrame(self.df_numeric_to_bins.transform(X[self.cols]), columns= self.cols).set_index(X.index)
        X[self.cols] = new_binned_cols.apply(lambda x: x.astype('str')+'_'+x.name)
        return X
    

class TheMightySelector (TransformerMixin, BaseEstimator):
    
    def __init__ (self, n= 10 ):
        super().__init__()
        self.n= n
        self.chi2_selector = SelectKBest(k=self.n, score_func= chi2)
        
    def fit(self, X, y=None):
        self.chi2_selector.fit(X, y)
        return self  
    
    def transform (self,X):
        self.k_cols= X.columns[self.chi2_selector.get_support(indices=False)]
        X_new= X[self.k_cols]
        #self.new= X_new
        return X_new
          
#%%

In [ ]:


X = df.drop(['Churn','customerID'], axis=1)
y= df['Churn']

X_train, X_test, y_train, y_test= train_test_split(X,y,test_size= 0.15, random_state= 12345)

cols1= ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
                        'OnlineSecurity', 'OnlineBackup','DeviceProtection', 'TechSupport', 'StreamingTV',   'StreamingMovies',
                       'Contract',  'PaperlessBilling', 'PaymentMethod']  

cols2= ['tenure', 'MonthlyCharges', 'TotalCharges']


#%%

from sklearn.ensemble import VotingClassifier
 
xgb= xgb.XGBClassifier()

rfc= RandomForestClassifier()

#lr= LogisticRegression()

classifiers= [("xgb",xgb), ("rfc", rfc)]

clf_voting= VotingClassifier(estimators=classifiers, voting='soft', weights=None)

  
#%%


final_model = Pipeline(steps=[('ohe1',MyOneHotEncoder(cat_cols=cols1)), 
                              ('rff',RandomForestFiller() ), 
                              ('be', BinsEncoder()), 
                              ('ohe2',MyOneHotEncoder(cat_cols=cols2)),
                              #('reduce_dim',SelectKBest(score_func= chi2)) ,
                              ('reduce_dim',TheMightySelector ()),
                              ('sampling', SMOTE()),
                              ('voter', clf_voting)])


#%%
params_store= final_model.get_params()


param_search = {
  'rff__max_depth' : [2],
  'be__k': [8],
  #'kbs__n': [3,12,23],
  'reduce_dim__n': [20,40],
  'voter__xgb__max_depth': [6],
  'voter__xgb__learning_rate': [0.004],
  'voter__xgb__gamma'           : [0.1],
  'voter__xgb__n_estimators': [125],
  'voter__rfc__class_weight': ['balanced'],
  'voter__rfc__max_depth':[4],
  'voter__xgb__subsample':[0.1],
  "voter__xgb__min_child_weight" : [1]
  }


scorer = {
    'precision_score': make_scorer(precision_score, average = 'macro')
}


gsearch = GridSearchCV(estimator=final_model,  cv=3, refit='precision_score',
                      scoring=scorer,   
                      param_grid=param_search, verbose=2 ) #n_jobs=4

gsearch.fit(X_train, y_train)

y_test_pred= gsearch.best_estimator_.predict_proba(X_test)
preds= gsearch.best_estimator_.predict(X_test)


cv_report= pd.DataFrame(gsearch.cv_results_) # gives accuracy score 

final_results= pd.DataFrame(classification_report(y_test, preds, output_dict=True))

# importances= pd.Series(gsearch.best_estimator_.named_steps["voter"].feature_importances_ , index=gsearch.best_estimator_.named_steps['ohe'].x_new.columns).sort_values(ascending=False)
conf_matrix= confusion_matrix(y_test ,preds)



new_cols= gsearch.best_estimator_.steps[4][1].k_cols

feauts= gsearch.best_estimator_.steps[6][1].estimators_[1].feature_importances_

pd.Series(feauts, index= new_cols).sort_values().idxmax()

#%%



In [ ]:

metric_graph(np.array(y_test=='Yes'), y_test_pred[:,1] ,metric='roc')


from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(np.array(y_test=='Yes'), y_test_pred[:,1], pos_label=1)
print("AUC=" , metrics.auc(fpr, tpr))
